In [1]:
from datetime import datetime
from utils.bronze_processor import generate_first_of_month_dates, process_bronze
from utils.spark import get_spark

if __name__ == "__main__":
    spark = get_spark("assignment1-bronze")

    df_dates = (
        spark.read.option("header", "true")
             .csv("data/raw/lms_loan_daily.csv")
             .selectExpr("min(snapshot_date) as min_dt", "max(snapshot_date) as max_dt")
             .collect()[0]
    )

    start_date_str = datetime.strptime(df_dates["min_dt"], "%Y-%m-%d").strftime("%Y-%m-01")
    end_date_str   = datetime.strptime(df_dates["max_dt"], "%Y-%m-%d").replace(day=1).strftime("%Y-%m-%d")

    dates = generate_first_of_month_dates(start_date_str, end_date_str)
    print(f"Back-filling Bronze for {len(dates)} months: {dates[0]} → {dates[-1]}")

    process_bronze(dates, spark)
    spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/15 13:43:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

NameError: name 'datetime' is not defined